In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from Scripts.def_colors import map_DMC, dmc_color
from Scripts.define_setup import *
from Scripts.myfit import fit_err, fun_lin, fun_quad, fun_cub, get_chi2_alpha_parfun
from jup_plot import *

# Set the display option for maximum rows (you can adjust this based on your needs)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Check if we are in Google Colab environment
try:
    import google.colab
    IN_COLAB = True
    usetex = False
except:
    import os
    IN_COLAB = False
    if os.path.expanduser('~') == '/home/shixubenjamin':
        usetex = True
    else:
        usetex = False

# If in Google Colab, install the necessary data and set up the necessary environment
if IN_COLAB == True:
    !rm -rf /content/DMC-reproducibility-main /content/main.zip
    !wget https://github.com/zenandrea/DMC-reproducibility/archive/refs/heads/main.zip
    !unzip /content/main.zip
    %pwd
    %cd /content/DMC-reproducibility-main

replot_graphs = True

# Load the data
dimer_info = pd.read_csv('Data/dim_info.csv', index_col=0)
monomer_info = pd.read_csv('Data/mol_info.csv', index_col=0)

dimer_dmc_total_energy_data = pd.read_csv('Data/results_dim.csv', index_col=0)
monomer_dmc_total_energy_data = pd.read_csv('Data/results_mol.csv', index_col=0)
monomer_geometry_correction_data = pd.read_csv('Data/delta_mol_ref.csv', index_col=0)

# Analysis of the DMC data for the S66 dataset

In [86]:
# Compute the binding energy for the S66
# Filter dmc data to only include data with dmc_type = 'DMCdla5' and dmc_Jas = 'Jopt'
filtered_dimer_dmc_total_energy_data = dimer_dmc_total_energy_data[(dimer_dmc_total_energy_data['dmc_type'] == 'DMCdla5') & (dimer_dmc_total_energy_data['dmc_Jas'] == 'Jopt')]

dmc_energy_data = {system_id: {'total_energy_dimer': 0, 'total_energy_monomer_1':0, 'total_energy_monomer_2':0, 'binding_energy': 0} for system_id in range(1,67)}


# Loop over the the dimers
for system_id, system_data in filtered_dimer_dmc_total_energy_data.groupby('ID'):
    system_data = system_data.sort_values('tau', ascending=False)
    system_data.set_index( 'tau', inplace=True )
    system_name = dimer_info.loc[system_id,'name']

    monomer_data = {1:0, 2:0}
    monomer_geometry_correction = {1:0, 2:0}
    # Get the monomer data
    for monomer_num in [1,2]:
        monomer_id = f'{system_id:02d}_{monomer_num}'
        monomer_name = dimer_info.loc[system_id,f'mol{monomer_num}']
        monomer_ref_id = monomer_info.loc[monomer_name, 'ref']
        monomer_ref_data = monomer_dmc_total_energy_data[(monomer_dmc_total_energy_data['mol_id'] == monomer_ref_id) & (monomer_dmc_total_energy_data['dmc_type'] == 'DMCdla5') & (monomer_dmc_total_energy_data['dmc_Jas'] == 'Jopt')].sort_values('tau', ascending=False)
        monomer_ref_data.set_index( 'tau', inplace=True )
        # Add the geometry correction
        monomer_ref_data['ene'] = monomer_ref_data['ene'] + monomer_geometry_correction_data[monomer_geometry_correction_data['mol_id'] == monomer_id]['ene-ref'].values[0]
        monomer_data[monomer_num] = monomer_ref_data
    dmc_energy_data[system_id]['total_energy_dimer'] = system_data.copy()
    dmc_energy_data[system_id]['total_energy_monomer_1'] = monomer_data[1]
    dmc_energy_data[system_id]['total_energy_monomer_2'] = monomer_data[2]
    # Compute the binding energy
    system_data['binding_energy'] = system_data['ene'] - monomer_data[1]['ene'] - monomer_data[2]['ene']
    system_data['binding_energy_err'] = (system_data['err']**2 + monomer_data[1]['err']**2 + monomer_data[2]['err']**2)**0.5
    dmc_energy_data[system_id]['binding_energy'] = system_data

## Analyzing previous CCSD(T) literature and generating CCSD(cT)-fit

In [87]:
keshwarni_cc_data = pd.read_excel('Data/Kesharwani_10.1071_CH17588_SI.xlsx', sheet_name='F12c_aVTZ-F12 CCSD',usecols = 'J:L').dropna().drop([7, 8]).reset_index(drop=True)
keshwarni_cc_data.columns = ['HF', 'MP2', 'CCSD']
keshwarni_cc_data['(T)'] = pd.read_excel('Data/Kesharwani_10.1071_CH17588_SI.xlsx', sheet_name='(T) ',usecols = 'P').drop(list(range(19))).reset_index(drop=True)['Unnamed: 15']
keshwarni_cc_data['(cT)'] = keshwarni_cc_data['(T)'] /( 0.7764+0.278*(keshwarni_cc_data['MP2'] - keshwarni_cc_data['HF'])/(keshwarni_cc_data['CCSD']  - keshwarni_cc_data['HF']))
keshwarni_cc_data['(cT)-(T)'] = keshwarni_cc_data['(cT)'] - keshwarni_cc_data['(T)']

# Load CCSD(T) references
ccsdt_references = pd.read_csv('Data/Refs.csv', index_col=0)
ccsdt_references['CCSD(T) Final'] = ccsdt_references['Hobza_1']
ccsdt_references['CCSD(T) Error'] = ccsdt_references['Hobza_1']

ccsdt_references['CCSD(cT)-fit Final'] = ccsdt_references['Hobza_1']

for i in range(66):
    if np.isnan(ccsdt_references['Martin_Gold'][i+1]):
        ccsdt_references.loc[i+1,'CCSD(T) Final'] = np.average([ccsdt_references['Hobza_2'][i+1], ccsdt_references['Martin_Silver'][i+1], ccsdt_references['14k-Gold'][i+1]])
        ccsdt_references.loc[i+1,'CCSD(T) Error'] = np.std([ccsdt_references['Hobza_2'][i+1], ccsdt_references['Martin_Silver'][i+1], ccsdt_references['14k-Gold'][i+1]])
        ccsdt_references.loc[i+1,'CCSD(cT)-fit Final'] = ccsdt_references.loc[i+1,'Martin_Silver'] - keshwarni_cc_data['(cT)-(T)'][i]

    else:
        ccsdt_references.loc[i+1,'CCSD(T) Final'] = np.average([ccsdt_references['Hobza_2'][i+1], ccsdt_references['Martin_Gold'][i+1], ccsdt_references['14k-Gold'][i+1]])
        ccsdt_references.loc[i+1,'CCSD(T) Error'] = np.std([ccsdt_references['Hobza_2'][i+1], ccsdt_references['Martin_Gold'][i+1], ccsdt_references['14k-Gold'][i+1]])
        ccsdt_references.loc[i+1,'CCSD(cT)-fit Final'] = ccsdt_references.loc[i+1,'Martin_Silver'] - keshwarni_cc_data['(cT)-(T)'][i]
    

## SI - Timestep dependence for the binding energy of each S66 system

In [99]:
dmc_energy_data[1]['binding_energy'].index.tolist()

[0.3,
 0.25,
 0.2,
 0.16,
 0.13,
 0.1,
 0.08,
 0.06,
 0.05,
 0.04,
 0.03,
 0.02,
 0.01,
 0.003,
 0.001]

In [97]:
dmc_energy_data[1]['binding_energy']['binding_energy'].index

Index([  0.3,  0.25,   0.2,  0.16,  0.13,   0.1,  0.08,  0.06,  0.05,  0.04,
        0.03,  0.02,  0.01, 0.003, 0.001],
      dtype='float64', name='tau')

In [139]:
 # Plot Binding Energy and 

final_binding_energy =  {f'{system_id}': [0,0] for system_id in range(1,67)}

if replot_graphs:
    for system_id in range(1,67):
        name = dimer_info.loc[system_id,'name']
        
        fig, ax = plt.subplots(1,2,figsize=(6.69,3) )
        ax[0].set_title( f'{name}' )
        ax[0].set_xlabel( 'DMC timestep [a.u.]' )
        ax[0].set_xticks( [0, 0.003, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.1, 0.2, 0.3 ] )
        ax[0].set_xticklabels( [ '0', '3E-3', '0.01', '0.02', '0.03', '0.04', '0.05', '0.06', '0.1', '0.2', '0.3' ], rotation=90 )
        ax[0].set_xlim( [0,0.1*1.03] )
        ax[0].set_ylabel( 'Binding Energy [kcal/mol]' )

        # reference quantum-chemistry result
        # ax[0].axhline( ccsdt_references.loc[system_id,'Hobza_1'], c='c', ls=':', label='Hobza' )
        ax[0].axhline( ccsdt_references.loc[system_id,'Hobza_2'], c='m', ls='-.', label='Hobza rev' )
        # ax[0].axhline( ccsdt_references.loc[system_id,'Martin_newBronze'], c='y', ls=':',  label='Martin newBronze' )
        ax[0].axhline( ccsdt_references.loc[system_id,'Martin_Silver'], c='silver', ls='-.',  label='Martin Silver' )
        # ax.axhline( ccsdt_references.loc[system_id,'Martin_PCCP2022'], c='k', ls='--', label='Martin 2022' )
        ax[0].axhline( ccsdt_references.loc[system_id,'14k-Gold'], c='gold', ls='--', label='Nagy 14k-Gold' )


        ax[0].errorbar(dmc_energy_data[system_id]['binding_energy'].index.tolist(), dmc_energy_data[system_id]['binding_energy']['binding_energy'].values, yerr=dmc_energy_data[system_id]['binding_energy']['binding_energy_err'].values, fmt='o', color='black',markeredgecolor='none',markersize=4, label=r'DMC//DLA')

        system_binding_energy_data = dmc_energy_data[system_id]['binding_energy']

        taumaxfit = 0.30 #0.10
        fitting_data = system_binding_energy_data[ system_binding_energy_data.index <= taumaxfit ]
        xdata = fitting_data.index.to_numpy()
        ydata = fitting_data['binding_energy'].to_numpy()
        sigma = fitting_data['binding_energy_err'].to_numpy()

        xfit, m, s = fit_err(xdata,ydata,sigma,fitfun=fun_quad)
        ax[0].plot(xfit,m,'--',color='red', label=r'quadratic fit ($E^\textrm{bind}_{\tau \to 0}=$' + f'{m[0]:.2f}' + r'${\pm}$' + f'{s[0]:.2f})')
        ax[0].fill_between(xfit,m-1*s,m+1*s,color='red',alpha=0.2)
        # print(f'{name}: limit tau->{xfit[0]} Eb = {m[0]:.2f} +/- {s[0]:.2f}')

        binding_energy_data = system_binding_energy_data[ system_binding_energy_data.index <= 0.045 ]
        xdata = binding_energy_data.index.to_numpy()
        ydata = binding_energy_data['binding_energy'].to_numpy()
        sigma = binding_energy_data['binding_energy_err'].to_numpy()

        xfit1, m1, s1 = fit_err(xdata,ydata,sigma,fitfun=fun_lin)
        ax[0].plot(xfit1,m1,'--',color='blue', label=r'linear fit ($E^\textrm{bind}_{\tau \to 0}=$' + f'{m1[0]:.2f}' + r'${\pm}$' + f'{s1[0]:.2f})')
        ax[0].fill_between(xfit1,m1-1*s1,m1+1*s1,color='blue',alpha=0.2)
        if abs(m[0] - m1[0]) > 0.1:
            print(f'lin {name}: {m[0]:.2f}({int(round(100*s[0]))}) {m1[0]:.2f}({int(round(100*s1[0]))}) {m[0] - m1[0]:.2f}')
            final_binding_energy[f'{system_id}'] = [m1[0],s1[0]]
        else:
            print(f'quad {name}: {m[0]:.2f}({int(round(100*s[0]))}) {m1[0]:.2f}({int(round(100*s1[0]))}) {m[0] - m1[0]:.2f}')
            final_binding_energy[f'{system_id}'] = [m[0],s[0]]



        ax[0].legend(fontsize=7)


        ax[1].set_title( f'{name}' )
        ax[1].set_xlabel( 'DMC timestep [a.u.]' )
        ax[1].set_xticks( [0, 0.003, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.1, 0.2, 0.3 ] )
        ax[1].set_xticklabels( [ '0', '3E-3', '0.01', '0.02', '0.03', '0.04', '0.05', '0.06', '0.1', '0.2', '0.3' ], rotation=90 )
        ax[1].set_xlim( [0,0.1*1.03] )
        ax[1].set_ylabel( 'Dimer Total Energy [kcal/mol]' )


        system_dimer_total_energy_data = dmc_energy_data[system_id]['total_energy_dimer']


        fitting_data = system_dimer_total_energy_data[ system_dimer_total_energy_data.index <= 0.045 ]
        xdata = fitting_data.index.to_numpy()
        ydata = fitting_data['ene'].to_numpy()
        sigma = fitting_data['err'].to_numpy()



        xfit1, m1, s1 = fit_err(xdata,ydata,sigma,fitfun=fun_lin)

        extrap_system_total_energy = m1[0]

        ax[1].plot(xfit1,m1 - extrap_system_total_energy,'--',color='blue', label=r'linear fit ($E^\textrm{bind}_{\tau \to 0}=$' + f'{m1[0]:.2f}' + r'${\pm}$' + f'{s1[0]:.2f})')
        ax[1].fill_between(xfit1,m1 - extrap_system_total_energy -1*s1,m1 - extrap_system_total_energy +1*s1,color='blue',alpha=0.2)

        ax[1].errorbar(dmc_energy_data[system_id]['total_energy_dimer'].index.tolist(), dmc_energy_data[system_id]['total_energy_dimer']['ene'].values - extrap_system_total_energy, yerr=dmc_energy_data[system_id]['total_energy_dimer']['err'].values, fmt='o', color='black',markeredgecolor='none',markersize=4, label=r'DMC//DLA')

        taumaxfit = 0.30 #0.10
        fitting_data = system_dimer_total_energy_data[ system_dimer_total_energy_data.index <= taumaxfit ]
        xdata = fitting_data.index.to_numpy()
        ydata = fitting_data['ene'].to_numpy()
        sigma = fitting_data['err'].to_numpy()

        xfit, m, s = fit_err(xdata,ydata,sigma,fitfun=fun_quad)
        ax[1].plot(xfit,m - extrap_system_total_energy,'--',color='red', label=r'quadratic fit ($E^\textrm{bind}_{\tau \to 0}=$' + f'{m[0]:.2f}' + r'${\pm}$' + f'{s[0]:.2f})')
        ax[1].fill_between(xfit,m - extrap_system_total_energy -1*s,m - extrap_system_total_energy +1*s,color='red',alpha=0.2)

        ax[1].legend(fontsize=7)

        ax[1].set_ylim([-15,15])

        fig.tight_layout()
        fig.savefig(f'Figures/Fig_SI_S66_{system_id:02d}.png',format='png',dpi=300)
        plt.close()
    
    np.save('Data/final_binding_energy.npy', final_binding_energy)

else:
    final_binding_energy = np.load('Data/final_binding_energy.npy', allow_pickle=True).item()

/tmp/ipykernel_5814/3807468315.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(1,2,figsize=(6.69,3) )


quad 01_Water-Water: -5.19(2) -5.17(2) -0.02


/tmp/ipykernel_5814/3807468315.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(1,2,figsize=(6.69,3) )


quad 02_Water-MeOH: -5.88(2) -5.86(3) -0.02


/tmp/ipykernel_5814/3807468315.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(1,2,figsize=(6.69,3) )


...

quad 64_Peptide-Ethene: -2.80(3) -2.77(5) -0.03


/home/shixubenjamin/Projects/DMC/Analysis/24_07_04-S66_Dataset/Scripts/myfit.py:38: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit( fitfun, xdata=xdata, ydata=ydata, sigma=ye )
/tmp/ipykernel_5814/3807468315.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(1,2,figsize=(6.69,3) )


quad 65_Pyridine-Ethyne: -4.26(3) -4.27(5) 0.01


/tmp/ipykernel_5814/3807468315.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(1,2,figsize=(6.69,3) )


quad 66_MeNH2-Pyridine: -3.88(3) -3.88(5) -0.01


In [134]:
extrap_system_total_energy

-5.167921990878764

In [117]:
final_binding_energy

{'1': [-5.186298365245017, 0.01779891687717257],
 '2': [0, 0],
 '3': [0, 0],
 '4': [0, 0],
 '5': [0, 0],
 '6': [0, 0],
 '7': [0, 0],
 '8': [0, 0],
 '9': [0, 0],
 '10': [0, 0],
 '11': [0, 0],
 '12': [0, 0],
 '13': [0, 0],
 '14': [0, 0],
 '15': [0, 0],
 '16': [0, 0],
 '17': [0, 0],
 '18': [0, 0],
 '19': [0, 0],
 '20': [0, 0],
 '21': [0, 0],
 '22': [0, 0],
 '23': [0, 0],
 '24': [0, 0],
 '25': [0, 0],
 '26': [0, 0],
 '27': [0, 0],
 '28': [0, 0],
 '29': [0, 0],
 '30': [0, 0],
 '31': [0, 0],
 '32': [0, 0],
 '33': [0, 0],
 '34': [0, 0],
 '35': [0, 0],
 '36': [0, 0],
 '37': [0, 0],
 '38': [0, 0],
 '39': [0, 0],
 '40': [0, 0],
 '41': [0, 0],
 '42': [0, 0],
 '43': [0, 0],
 '44': [0, 0],
 '45': [0, 0],
 '46': [0, 0],
 '47': [0, 0],
 '48': [0, 0],
 '49': [0, 0],
 '50': [0, 0],
 '51': [0, 0],
 '52': [0, 0],
 '53': [0, 0],
 '54': [0, 0],
 '55': [0, 0],
 '56': [0, 0],
 '57': [0, 0],
 '58': [0, 0],
 '59': [0, 0],
 '60': [0, 0],
 '61': [0, 0],
 '62': [0, 0],
 '63': [0, 0],
 '64': [0, 0],
 '65': [0, 0],

In [112]:
'linear fit ' + r'$E_^\textrm{bind}_{\tau \to 0}$' + '({m1[0]:.2f}' + r'${\pm}$' + f'{s1[0]:.2f})'

'linear fit $E_^\\textrm{bind}_{\\tau \\to 0}$({m1[0]:.2f}${\\pm}$0.02)'

## Main - Comparison of DMC against CCSD(T) and CCSD(cT)

In [74]:
# Print the final data. Create a figure with three rows and plot final_binding_energy
fig, axs = plt.subplots( nrows=3, ncols=1, figsize=(6.67,7),dpi=600,constrained_layout=True)

datarange1 = list(range(1,24))
datarange2 = list(range(24,47))
datarange3 = list(range(47,67))

axs[0].axhline(0, color='k', ls='--')
axs[1].axhline(0, color='k', ls='--')
axs[2].axhline(0, color='k', ls='--')


axs[0].errorbar(datarange1,[final_binding_energy[f'{i}'][0] - final_binding_energy[f'{i}'][0] for i in datarange1], yerr = [final_binding_energy[f'{i}'][1] for i in datarange1], capsize=3, marker = 'none', ls='none', color = 'blue')
axs[1].errorbar(datarange2,[final_binding_energy[f'{i}'][0] - final_binding_energy[f'{i}'][0] for i in datarange2],yerr = [final_binding_energy[f'{i}'][1] for i in datarange2], marker = 'none',ls='none', color = 'blue', capsize=3)
axs[2].errorbar(datarange3,[final_binding_energy[f'{i}'][0] - final_binding_energy[f'{i}'][0] for i in datarange3], yerr = [final_binding_energy[f'{i}'][1] for i in datarange3], marker = 'none',ls='none', color = 'blue', capsize=3)

# Plot the Martin silver reference value
axs[0].scatter(datarange1, [ccsdt_references.loc[x,'CCSD(T) Final'] - final_binding_energy[f'{x}'][0] for x in datarange1],c='silver',marker='x', label=f'CCSD(T) [MAD: {np.mean([abs(ccsdt_references.loc[x,"CCSD(T) Final"] - final_binding_energy[f"{x}"][0]) for x in datarange1]):.2f}]')
axs[1].scatter(datarange2, [ccsdt_references.loc[x,'CCSD(T) Final'] - final_binding_energy[f'{x}'][0] for x in datarange2],c='silver',marker='x', label=f'CCSD(T) [MAD: {np.mean([abs(ccsdt_references.loc[x,"CCSD(T) Final"] - final_binding_energy[f"{x}"][0]) for x in datarange2]):.2f}]')
axs[2].scatter(datarange3, [ccsdt_references.loc[x,'CCSD(T) Final'] - final_binding_energy[f'{x}'][0] for x in datarange3],c='silver',marker='x', label=f'CCSD(T) [MAD: {np.mean([abs(ccsdt_references.loc[x,"CCSD(T) Final"] - final_binding_energy[f"{x}"][0]) for x in datarange3]):.2f}]')

# Plot the cT_data
# axs[0].scatter(datarange1, [-s66_cT_data[x-1]- final_binding_energy[f'{x}'][0] for x in datarange1],c='gold',marker='x', label=f'CCSD(cT)-fit [MAD: {np.mean([abs(-s66_cT_data[x-1] - final_binding_energy[f"{x}"][0]) for x in datarange1]):.2f}]')
axs[1].scatter(datarange2, [ccsdt_references.loc[x,'CCSD(cT)-fit Final']- final_binding_energy[f'{x}'][0] for x in datarange2],c='gold',marker='x', label=f'CCSD(cT)-fit [MAD: {np.mean([abs(ccsdt_references.loc[x,"CCSD(cT)-fit Final"] - final_binding_energy[f"{x}"][0]) for x in datarange2]):.2f}]')
# axs[2].scatter(datarange3, [-s66_cT_data[x-1]- final_binding_energy[f'{x}'][0] for x in datarange3],c='gold',marker='x', label=f'CCSD(cT)-fit [MAD: {np.mean([abs(-s66_cT_data[x-1] - final_binding_energy[f"{x}"][0]) for x in datarange3]):.2f}]')



axs[0].set_xticks(datarange1)
# Plot the names in the figure
for i in datarange1:
    axs[0].text(i,-0.9,f"{final_binding_energy[f'{i}'][0]:.2f}({int(round(100*final_binding_energy[f'{i}'][1]))})",fontsize=8,ha='center',rotation=90,  bbox=dict(facecolor='white', edgecolor='none',alpha=0.8 ))

axs[1].set_xticks(datarange2)
for i in datarange2:
    axs[1].text(i,-0.9,f"{final_binding_energy[f'{i}'][0]:.2f}({int(round(100*final_binding_energy[f'{i}'][1]))})",fontsize=8,ha='center',rotation=90,  bbox=dict(facecolor='white', edgecolor='none',alpha=0.8 ))
axs[2].set_xticks(datarange3)
for i in datarange3:
    axs[2].text(i,-0.9,f"{final_binding_energy[f'{i}'][0]:.2f}({int(round(100*final_binding_energy[f'{i}'][1]))})",fontsize=8,ha='center',rotation=90,  bbox=dict(facecolor='white', edgecolor='none',alpha=0.8 ))

axs[0].set_ylim([-1,1])
axs[1].set_ylim([-1,1])
axs[2].set_ylim([-1,1])

axs[2].set_xlabel('S66 system')

axs[0].legend(loc='upper left')
axs[1].legend(loc='upper left')
axs[2].legend(loc='upper left')

axs[0].set_title('H-bonded systems')
axs[1].set_title('Dispersion systems')
axs[2].set_title('Mixed systems')

fig.supylabel('Difference against DMC [kcal/mol]')

plt.savefig('Figures/Fig_MAIN_S66_compare.png')

In [82]:
# Plot relative differences for all systems
fig, axs = plt.subplots( nrows=3, ncols=1, figsize=(6.67,7),dpi=600,constrained_layout=True)

datarange1 = list(range(1,24))
datarange2 = list(range(24,47))
datarange3 = list(range(47,67))

axs[0].axhline(0, color='k', ls='--')
axs[1].axhline(0, color='k', ls='--')
axs[2].axhline(0, color='k', ls='--')


axs[0].errorbar(datarange1,[final_binding_energy[f'{i}'][0] - final_binding_energy[f'{i}'][0] for i in datarange1], yerr = [abs(final_binding_energy[f'{i}'][1]*100/final_binding_energy[f'{i}'][0]) for i in datarange1], capsize=3, marker = 'none', ls='none', color = 'blue')
axs[1].errorbar(datarange2,[final_binding_energy[f'{i}'][0] - final_binding_energy[f'{i}'][0] for i in datarange2],yerr = [abs(final_binding_energy[f'{i}'][1]*100/final_binding_energy[f'{i}'][0]) for i in datarange2], marker = 'none',ls='none', color = 'blue', capsize=3)
axs[2].errorbar(datarange3,[final_binding_energy[f'{i}'][0] - final_binding_energy[f'{i}'][0] for i in datarange3], yerr = [abs(final_binding_energy[f'{i}'][1]*100/final_binding_energy[f'{i}'][0]) for i in datarange3], marker = 'none',ls='none', color = 'blue', capsize=3)

# Plot the final (averaged) CCSD(T) reference value
axs[0].scatter(datarange1, [(ccsdt_references.loc[x,'CCSD(T) Final'] - final_binding_energy[f'{x}'][0])*100/final_binding_energy[f'{x}'][0] for x in datarange1],c='silver',marker='x', label=f'CCSD(T) [MRD: {np.mean([abs((ccsdt_references.loc[x,"CCSD(T) Final"] - final_binding_energy[f"{x}"][0])/final_binding_energy[f"{x}"][0])*100 for x in datarange1]):.2f}%]')
axs[1].scatter(datarange2, [(ccsdt_references.loc[x,'CCSD(T) Final'] - final_binding_energy[f'{x}'][0])*100/final_binding_energy[f'{x}'][0] for x in datarange2],c='silver',marker='x', label=f'CCSD(T) [MRD: {np.mean([abs((ccsdt_references.loc[x,"CCSD(T) Final"] - final_binding_energy[f"{x}"][0])/final_binding_energy[f"{x}"][0])*100 for x in datarange2]):.2f}%]')
axs[2].scatter(datarange3, [(ccsdt_references.loc[x,'CCSD(T) Final'] - final_binding_energy[f'{x}'][0])*100/final_binding_energy[f'{x}'][0] for x in datarange3],c='silver',marker='x', label=f'CCSD(T) [MRD: {np.mean([abs((ccsdt_references.loc[x,"CCSD(T) Final"] - final_binding_energy[f"{x}"][0])/final_binding_energy[f"{x}"][0])*100 for x in datarange3]):.2f}%]')

# Plot the cT_data
# axs[0].scatter(datarange1, [(-s66_cT_data[x-1]- final_binding_energy[f'{x}'][0])*100/final_binding_energy[f'{x}'][0] for x in datarange1],c='gold',marker='x', label=f'CCSD(cT)-fit [MAD: {np.mean([abs(-s66_cT_data[x-1] - final_binding_energy[f"{x}"][0]) for x in datarange1]):.2f}]')
axs[1].scatter(datarange2, [(ccsdt_references.loc[x,'CCSD(cT)-fit Final']- final_binding_energy[f'{x}'][0])*100/final_binding_energy[f'{x}'][0] for x in datarange2],c='gold',marker='x', label=f'CCSD(cT)-fit [MRD: {np.mean([abs((ccsdt_references.loc[x,"CCSD(cT)-fit Final"] - final_binding_energy[f"{x}"][0])/final_binding_energy[f"{x}"][0])*100 for x in datarange2]):.2f}%]')
# axs[2].scatter(datarange3, [(-s66_cT_data[x-1]- final_binding_energy[f'{x}'][0])*100/final_binding_energy[f'{x}'][0] for x in datarange3],c='gold',marker='x', label=f'CCSD(cT)-fit [MAD: {np.mean([abs(-s66_cT_data[x-1] - final_binding_energy[f"{x}"][0]) for x in datarange3]):.2f}]')

axs[0].set_xticks(datarange1)
axs[1].set_xticks(datarange2)
axs[2].set_xticks(datarange3)

axs[0].set_ylim([-10,20])
axs[1].set_ylim([-10,20])
axs[2].set_ylim([-10,20])

axs[2].set_xlabel('S66 system')
axs[0].legend(loc='upper center')
axs[1].legend(loc='upper center')
axs[2].legend(loc='upper center')

axs[0].set_title('H-bonded systems')
axs[1].set_title('Dispersion systems')
axs[2].set_title('Mixed systems')

fig.supylabel('Relative difference against DMC \%')
plt.savefig('Figures/Fig_SI_S66_compare_relative.png')

In [ ]:
# # Make the data for the dictionary of the 

# with open('s66_rel_ene.txt','w') as f:
#     f.write("System DMC_Int_Ene (T)_Int_Ene (cT)_Int_Ene (T)_Diff (cT)_Diff (T)_Rel_Diff (cT)_Rel_Diff\n")
#     for i in range(1,67):
#         f.write(f'{i:02d}      {final_binding_energy[f"{i}"][0]:-7.3f}    {ccsdt_references.loc[i,"Martin_Silver"]:-7.3f}     {-s66_cT_data[i-1]:-7.3f}     {(ccsdt_references.loc[i,"Martin_Silver"] -  final_binding_energy[f"{i}"][0]):-7.3f}    {(-s66_cT_data[i-1]- final_binding_energy[f"{i}"][0]):-7.3f}     {abs((ccsdt_references.loc[i,"Martin_Silver"] - final_binding_energy[f"{i}"][0])*100/final_binding_energy[f"{i}"][0]):7.1f}         {abs((-s66_cT_data[i-1]- final_binding_energy[f"{i}"][0])*100/final_binding_energy[f"{i}"][0]):4.1f}\n')


## Analysis of differences based on SAPT

In [84]:
# Plot the error between DMC and CCSD(T) against the dispersion/electrostatic ratio

import Data.Sherrill_Biofragment_SAPT_S66 as sapt_s66

binding_energy_decomposition = pd.DataFrame( sapt_s66.DATA )

binding_energy_decomposition['ELST DISP+ELST RATIO'] = binding_energy_decomposition['SAPT ELST ENERGY'] /(binding_energy_decomposition['SAPT DISP ENERGY'] + binding_energy_decomposition['SAPT ELST ENERGY'])


fig, axs = plt.subplots(figsize=(3.36,3.5),dpi=600,constrained_layout=True)

quantity_to_look_at = 'ELST DISP+ELST RATIO'

axs.scatter(np.array(binding_energy_decomposition[quantity_to_look_at].tolist())[[x-1 for x in datarange1]], [(ccsdt_references.loc[x,'Martin_Silver'] - final_binding_energy[f'{x}'][0])*100/final_binding_energy[f'{x}'][0] for x in datarange1],c='red',marker='x', label='Electrostatic')
axs.scatter(np.array(binding_energy_decomposition[quantity_to_look_at].tolist())[[x-1 for x in datarange2]], [(ccsdt_references.loc[x,'Martin_Silver'] - final_binding_energy[f'{x}'][0])*100/final_binding_energy[f'{x}'][0] for x in datarange2],c='blue',marker='x',label='Dispersion')
axs.scatter(np.array(binding_energy_decomposition[quantity_to_look_at].tolist())[[x-1 for x in datarange3]], [(ccsdt_references.loc[x,'Martin_Silver'] - final_binding_energy[f'{x}'][0])*100/final_binding_energy[f'{x}'][0] for x in datarange3],c='green',marker='x', label='Mixed')

axs.set_xlabel(r'ELST/DISP ratio from SAPT')
axs.set_ylabel(r'|DMC-CCSD(T)|/|DMC| [%]')
axs.legend()

plt.savefig('Figures/Fig_MAIN_Error_decomposition.png')
